In [25]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='pcs7_day7', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pcs7_day7...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:25


In [24]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [19]:
pcs6_start_date = "2022-03-16"
pcs6_end_date = "2022-03-22"
pcs6_gcoin = load_data_mart("pc", pcs6_start_date, pcs6_end_date, "gcoin_use").where(col('event_name').like("%pcs%"))

In [20]:
pcs6_gcoin = pcs6_gcoin.withColumn("paid_pu", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None)))

In [21]:
pcs6_gcoin.select(sum("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), countDistinct("paid_pu").alias("paid_pu"), countDistinct("account_id").alias("total_pu")).show(truncate=False)

+-------------+---------+-------+--------+
|sum(paid_use)|total_use|paid_pu|total_pu|
+-------------+---------+-------+--------+
|74265946     |109324730|41456  |62372   |
+-------------+---------+-------+--------+



In [22]:
pcs6_gcoin.groupBy("product_name").agg(sum("qty").alias("unit_sold"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), countDistinct("paid_pu").alias("paid_pu"), countDistinct("account_id").alias("total_pu")).toPandas()

,product_name,unit_sold,paid_use,total_use,paid_pu,total_pu
0,PCS6 PARACHUTE,6755,3783141,6687450,4850,6755
1,PCS6 - TRIUMPH & ANGUISH DANCE,9508,2303690,4754000,5331,9508
2,PCS6 BRUSHSTROKE MINI14,6770,2800985,6702300,4060,6770
3,PCS6 BRUSHSTROKE K2,2365,1240850,2341350,1644,2365
4,PCS6 BRUSHSTROKE CLOTHING SET,17660,6341460,17483400,8504,17660
5,PCS6 WEAPON & PARACHUTE BUNDLE,8206,12270505,16247880,7515,8206
6,PCS6 MEGA BUNDLE,11679,41152475,46248840,11388,11679
7,PCS6 GEAR BUNDLE,8949,4372840,8859510,5707,8949


In [3]:
start_date = "2022-09-09"
end_date = "2022-09-15"

In [4]:
gcoin = load_data_mart('pc', start_date, end_date, "gcoin_use")
pcs_gcoin = gcoin.where(col('event_name').like("%pcs%"))

In [5]:
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

user = load_data_mart("pc", end_date, end_date, "user_master")
user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left").withColumnRenamed("accountid", "account_id")

In [28]:
console_start_date = "2022-09-15"
console_end_date = "2022-09-21"

In [31]:
console_pcs_gcoin = load_data_mart('console', console_start_date, console_end_date, "gcoin_use").where(col("event_name").like("%pcs%"))

In [32]:
console_pcs_gcoin.select(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show()

+--------+---------+
|paid_use|total_use|
+--------+---------+
| 6376760|  7990380|
+--------+---------+



In [34]:
console_user = load_data_mart("console", console_end_date, console_end_date, "user_master").where(col("lastlogindate") >= console_start_date)

In [35]:
console_user.select(countDistinct("accountid").alias("au")).show()

+------+
|    au|
+------+
|621588|
+------+



In [6]:
user.where(col('lastlogindate')>=start_date).select(countDistinct("account_id").alias("wau")).show()

+-------+
|    wau|
+-------+
|5133445|
+-------+



In [14]:
user.where(col('lastlogindate')>=start_date).groupBy("pubg_region").agg(countDistinct("account_id").alias("wau")).toPandas()

,pubg_region,wau
0,CN,3366337
1,NA,119054
2,SA,106954
3,KR,606733
4,SEA,211994
5,Undefined,189
6,JP,14209
7,TW/HK,28739
8,CIS,312665
9,EMEA,346842


In [9]:
pcs_gcoin = pcs_gcoin.join(user.select("account_id", "pubg_region"), "account_id").withColumn("paid_pu", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None)))

In [12]:
pcs_gcoin.groupBy("pubg_region").agg(countDistinct("paid_pu").alias("paid_pu"), countDistinct("account_id").alias("total_pu"), \
                                        sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()

,pubg_region,paid_pu,total_pu,paid_use,total_use
0,CN,22558,29422,34750315,48819610
1,NA,829,1035,1312370,1743920
2,SA,370,606,466690,834860
3,KR,5694,7214,8772370,11484070
4,SEA,682,993,944910,1508030
5,JP,84,104,131010,182300
6,TW/HK,211,266,347080,468690
7,CIS,852,1371,1129130,1784310
8,EMEA,1614,2241,2346470,3400970
9,OC,173,225,282980,369040


In [15]:
pcs_gcoin.groupBy("product_name").agg(sum("qty").alias("unit_sold"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), countDistinct("paid_pu").alias("paid_pu"), countDistinct("account_id").alias("total_pu")).toPandas()

,product_name,unit_sold,paid_use,total_use,paid_pu,total_pu
0,PCS7 CLOTHING BUNDLE,6781,3598875,6713190,4497,6781
1,PCS7 GEAR SET,13890,7580795,13751100,9743,13890
2,PCS7 PARACHUTE PACK,1850,1068240,1831500,1354,1850
3,PCS7 MK12 PACK,4278,2749920,4235220,3373,4278
4,PCS7 EMOTE PACK,7007,1813720,3503500,4191,7007
5,PCS7 SKS PACK,1317,658790,1303830,866,1317
6,PCS7 MEGA BUNDLE,11912,29987385,35378640,11612,11912
7,PCS7 WEAPON + PARACHUTE SET,1959,3025600,3878820,1838,1959
